In [2]:
!nvidia-smi
from tensorflow.keras import layers
import tensorflow as tf

Mon Mar  4 08:17:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0              27W /  70W |      2MiB / 15360MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

2024-03-04 08:17:30.395952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 08:17:30.541049: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-03-04 08:17:30.541178: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-03-04 08:17:30.541387: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-0

In [3]:
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

In [4]:
#!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

In [5]:
#unzip_data('nlp_getting_started.zip')

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state = 42)

In [8]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
len(train_df),len(test_df)

(7613, 3263)

In [12]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target : {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text : \n{text}\n")
    print("----\n")
    


Target : 0 (not real disaster)
Text : 
The way you move is like a full on rainstorm and I'm a house of cards.

----

Target : 1 (real disaster)
Text : 
Newlyweds feed thousands of Syrian refugees instead of hosting a banquet wedding dinner -  http://t.co/XZV0lT9ZZk via @smh

----

Target : 0 (not real disaster)
Text : 
Is This Country Latin America's Next 'Argentina': One week ago we reported on the economic devastation in he o... http://t.co/m2y9Ym3iF6

----

Target : 1 (real disaster)
Text : 
THIS IS A ONE DIRECTION CONCERT NOT A WAR ZONE WHAT IS THIS #OTRAMETLIFE http://t.co/PtY9HRCUZH

----

Target : 1 (real disaster)
Text : 
11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...

----



In [13]:
#007 Splitting data into training and validation sets.mp4
from sklearn.model_selection import train_test_split
train_sentences , val_sentences ,train_labels,val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)
                                                                            
                                                                            

In [14]:
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [15]:
#008 Converting text data to numbers using tokenisation and embeddings (overview).mp4
#009 Setting up a TensorFlow TextVectorization layer to convert text to numbers.mp4
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#OOV  = "Out of Vocabulary"
text_vectorizer = TextVectorization(max_tokens=10000,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)
                                    
                                    
                                    

2024-03-04 08:17:37.639671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-04 08:17:37.647735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-04 08:17:37.649706: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-04 08:17:37.651852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebu

In [16]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length,
                                   )

In [18]:
#010 Mapping the TextVectorization layer to text data and turning it into numbers
# Fit the text vectorizer to training text
text_vectorizer.adapt(train_sentences)

In [19]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 My first staining attempt was a disaster https://t.co/buDmKE3nNf\n
Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  13,   97, 8142, 6170,   23,    3,   75,    1,    0,    0,    0,
           0,    0,    0,    0]])>

In [20]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
#[UNK] meand unknown token

In [21]:
len(words_in_vocab)

10000

In [22]:
#011 Creating an Embedding layer to turn tokenised text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length = max_length
                             )

In [23]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/lEYJwNnAH8\n
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04290896,  0.04410477,  0.04971582, ..., -0.03619032,
         -0.0078494 , -0.02844566],
        [-0.04500362,  0.03126596, -0.02693068, ..., -0.04747234,
         -0.00548991, -0.00104075],
        [-0.00256319,  0.04143794,  0.03228917, ...,  0.04919317,
          0.02227721,  0.01602114],
        ...,
        [-0.0352472 , -0.00723988, -0.04660585, ..., -0.02646844,
         -0.02148832, -0.01987053],
        [ 0.03187854,  0.04614352, -0.03648425, ...,  0.01759586,
         -0.01472127, -0.04928131],
        [-0.02414148,  0.04470484, -0.03144456, ..., -0.01529334,
         -0.019906  , -0.01460209]]], dtype=float32)>

In [24]:
#check out a single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-4.29089554e-02,  4.41047661e-02,  4.97158207e-02, -2.46787909e-02,
         2.06654184e-02, -2.97703277e-02, -3.53503972e-04, -2.30071694e-03,
         4.41252254e-02,  2.06075646e-02,  2.42041014e-02,  2.98861377e-02,
         2.18346231e-02, -2.54189856e-02,  9.50048119e-03, -1.08846910e-02,
         2.04252340e-02,  4.90413196e-02,  1.16344541e-03, -4.77046371e-02,
        -8.98499414e-03, -1.87143907e-02, -2.21037269e-02, -2.39519719e-02,
        -1.37994178e-02,  4.62324284e-02, -2.16569658e-02,  1.32397562e-03,
         2.78112628e-02, -3.06096319e-02,  3.42944376e-02, -1.97826382e-02,
        -1.51039846e-02, -1.21294372e-02,  1.75222866e-02,  3.97433378e-02,
         1.53800733e-02, -5.02052158e-03,  4.62375991e-02, -4.11792770e-02,
        -3.44661474e-02,  1.61793716e-02,  3.87888029e-03, -2.65776645e-02,
        -4.05302532e-02, -5.02067804e-03,  3.62547301e-02, -4.62509058e-02,
        -3.17115709e-03, -4.53123711e-0

In [25]:
#012 Discussing the various modelling experiments we're going to run
#013 Model 0 Building a baseline model to try and improve upon
#Multinomial naive bais using TF_IDF formula
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),

])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
#Evalute our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score*100:.2f}%")

Our baseline model achives an accuracy of : 79.27%


In [27]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [28]:
#014 Creating a function to track and evaluate our model's results
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred)*100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true, y_pred,average="weighted")
    model_results = {"accuracy":model_accuracy,
                     "precision":model_precision,
                     "recall":model_recall,
                     "f1":model_f1}
    return model_results

In [29]:
baseline_results = calculate_results(y_true=val_labels,y_pred=baseline_preds)
baseline_results
#higher precision leads to less false positives
#higher recall leads to less false negetives

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [30]:
#015 Model 1 Building, fitting and evaluating our first deep model on text data
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers
SAVE_DIR = "model_logs"
#build model with Function API
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x) #layers.Reshape((1920,))(x) #.
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")
model_1.summary()



Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [31]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240304-081741
Epoch 1/5


2024-03-04 08:17:43.042044: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x5611b60ff8d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-04 08:17:43.042089: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-03-04 08:17:43.047459: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-04 08:17:43.309562: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


215/215 [==============================] - 13s 52ms/step - loss: 0.6121 - accuracy: 0.6863 - val_loss: 0.5361 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4416 - accuracy: 0.8190 - val_loss: 0.4693 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3471 - accuracy: 0.8608 - val_loss: 0.4597 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2851 - accuracy: 0.8908 - val_loss: 0.4624 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2378 - accuracy: 0.9115 - val_loss: 0.4781 - val_accuracy: 0.7822


In [32]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4781 - accuracy: 0.7822


[0.4781489372253418, 0.7821522355079651]

In [33]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0]

24/24 [==============================] - 0s 2ms/step


array([0.35431677], dtype=float32)

In [34]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [35]:
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [36]:
#calculate model 1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7860403042839984,
 'recall': 0.7821522309711286,
 'f1': 0.779380517102918}

In [37]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [38]:
#016 Visualising our model's learned word embeddings with TensorFlow's projector tool
words_in_vocab = text_vectorizer.get_vocabulary()

In [39]:
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [40]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [41]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape
#https://projector.tensorflow.org/
#https://www.tensorflow.org/text/guide/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk

(10000, 128)

In [42]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [43]:
#017 High-level overview of Recurrent Neural Networks (RNNs) + where to learn more
#018 Model 2 Building, fitting and evaluating our first TensorFlow RNN model (LSTM)
#MOdel 2 LSTM
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")


In [44]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [45]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240304-081805
Epoch 1/5


2024-03-04 08:18:08.285842: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8204


215/215 [==============================] - 12s 41ms/step - loss: 0.2209 - accuracy: 0.9234 - val_loss: 0.5988 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1584 - accuracy: 0.9421 - val_loss: 0.5749 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.1288 - accuracy: 0.9529 - val_loss: 0.6843 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1058 - accuracy: 0.9609 - val_loss: 0.7979 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0850 - accuracy: 0.9653 - val_loss: 0.9645 - val_accuracy: 0.7703


In [46]:
#make prediction with LSTM Model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[5.1919013e-02],
       [7.8006774e-01],
       [9.9957019e-01],
       [8.9544002e-03],
       [5.2793464e-04],
       [9.9828750e-01],
       [9.6063989e-01],
       [9.9970275e-01],
       [9.9956435e-01],
       [4.5988250e-01]], dtype=float32)

In [47]:
model_2_pred=tf.squeeze(tf.round(model_2_pred_probs))

In [48]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_pred)

In [49]:
model_2_results

{'accuracy': 77.03412073490814,
 'precision': 0.7726339681635592,
 'recall': 0.7703412073490814,
 'f1': 0.7679241301282296}

In [50]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [51]:
#019 Model 3 Building, fitting and evaluating a GRU-cell powered RNN
#Gated Recurrent Unit
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x= layers.LSTM(64,return_sequences=True)(x)
# x = layers.GRU(64)(x)
# x= layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [52]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                       

In [53]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240304-081826
Epoch 1/5
215/215 [==============================] - 11s 38ms/step - loss: 0.1530 - accuracy: 0.9418 - val_loss: 0.8422 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0817 - accuracy: 0.9707 - val_loss: 0.9089 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0718 - accuracy: 0.9734 - val_loss: 1.0620 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0599 - accuracy: 0.9758 - val_loss: 1.1150 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0536 - accuracy: 0.9768 - val_loss: 1.2823 - val_accuracy: 0.7703


In [54]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_pred)
model_3_results

24/24 [==============================] - 0s 3ms/step


{'accuracy': 77.03412073490814,
 'precision': 0.7711671866902318,
 'recall': 0.7703412073490814,
 'f1': 0.7686901866564684}

In [55]:
model_2_results

{'accuracy': 77.03412073490814,
 'precision': 0.7726339681635592,
 'recall': 0.7703412073490814,
 'f1': 0.7679241301282296}

In [60]:
#020 Model 4 Building, fitting and evaluating a bidirectional RNN model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x= layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [61]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 15, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                             

In [64]:
model_4.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])
            

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240304-095325
Epoch 1/5
215/215 [==============================] - 17s 46ms/step - loss: 0.1012 - accuracy: 0.9691 - val_loss: 1.0333 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0558 - accuracy: 0.9769 - val_loss: 1.2535 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0463 - accuracy: 0.9793 - val_loss: 1.2221 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0422 - accuracy: 0.9804 - val_loss: 1.4950 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0421 - accuracy: 0.9791 - val_loss: 1.5481 - val_accuracy: 0.7717


In [66]:
model_4_pred_probs = model_4.predict(val_sentences)

24/24 [==============================] - 1s 5ms/step


In [68]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [69]:
model_4_results = calculate_results(y_true=val_labels,y_pred = model_4_preds)

In [70]:
model_4_results

{'accuracy': 77.16535433070865,
 'precision': 0.773049502603257,
 'recall': 0.7716535433070866,
 'f1': 0.7697096359746837}

In [71]:
model_3_results

{'accuracy': 77.03412073490814,
 'precision': 0.7711671866902318,
 'recall': 0.7703412073490814,
 'f1': 0.7686901866564684}

In [78]:
#021 Discussing the intuition behind Conv1D neural networks for text and sequences
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters = 64,
                        kernel_size = 5,
                        activation="relu",
                        padding = "same")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape ,conv_1d_output.shape , max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 64]), TensorShape([1, 64]))

In [79]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02086991,  0.08165911, -0.07756181, ...,  0.02898308,
         -0.02571582,  0.04790627],
        [-0.0091746 ,  0.01617068,  0.05305117, ...,  0.03256897,
         -0.04507973,  0.0544344 ],
        [ 0.03355481,  0.02957534, -0.04665152, ..., -0.01679854,
         -0.03610291, -0.07688629],
        ...,
        [ 0.00037356,  0.00934782, -0.02627644, ...,  0.00188377,
         -0.01557949,  0.00767225],
        [ 0.00037356,  0.00934782, -0.02627644, ...,  0.00188377,
         -0.01557949,  0.00767225],
        [ 0.00037356,  0.00934782, -0.02627644, ...,  0.00188377,
         -0.01557949,  0.00767225]]], dtype=float32)>

In [80]:
max_pool_output

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[0.02390051, 0.05817342, 0.05397944, 0.05006459, 0.04131936,
        0.03641652, 0.03361323, 0.08620346, 0.06621982, 0.05934235,
        0.04523626, 0.05142265, 0.0390342 , 0.04901423, 0.03124667,
        0.04304642, 0.06344274, 0.0384756 , 0.01929648, 0.04705708,
        0.        , 0.03468434, 0.05976484, 0.02937589, 0.10519412,
        0.02145461, 0.        , 0.08680236, 0.05905075, 0.05111921,
        0.06495804, 0.04359585, 0.06184407, 0.07084122, 0.07285683,
        0.0374371 , 0.00753896, 0.0339309 , 0.02826807, 0.05651209,
        0.07565297, 0.02430327, 0.0385054 , 0.04918174, 0.05467587,
        0.02087993, 0.01696311, 0.03223751, 0.03212143, 0.03787361,
        0.11931382, 0.06679748, 0.03755444, 0.07266301, 0.01061253,
        0.09894377, 0.04512641, 0.08601889, 0.07652764, 0.03538071,
        0.03182619, 0.04467126, 0.06930812, 0.022138  ]], dtype=float32)>

In [87]:
#022 Model 5 Building, fitting and evaluating a 1D CNN for text
from tensorflow.keras import layers
input = layers.Input(shape=(1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,kernel_size=5,strides = 1,activation = "relu",padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name="model_5_Conv1D")

model_5.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [88]:
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_4 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_4 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [89]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20240304-115500
Epoch 1/5
215/215 [==============================] - 9s 36ms/step - loss: 0.1256 - accuracy: 0.9588 - val_loss: 0.8925 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0754 - accuracy: 0.9711 - val_loss: 1.0369 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0612 - accuracy: 0.9766 - val_loss: 1.1590 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0554 - accuracy: 0.9769 - val_loss: 1.2052 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0505 - accuracy: 0.9797 - val_loss: 1.2271 - val_accuracy: 0.7533


In [90]:
model_5_pred_probs = model_5.predict(val_sentences)

24/24 [==============================] - 0s 2ms/step


In [91]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

In [92]:
model_5_results = calculate_results(y_true = val_labels,
                                    y_pred = model_5_preds,)
model_5_results

{'accuracy': 75.32808398950131,
 'precision': 0.7542297272597304,
 'recall': 0.7532808398950132,
 'f1': 0.7511805262255202}

In [93]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
#023 Using TensorFlow Hub for pretrained word embeddings (transfer learning for NLP)
